In [1]:
import jax
import torch
import numpy
from huggingface_hub import hf_hub_download
import jax

from moshi.moshi.models.loaders import get_mimi
from moshi_jax.moshi_jax.models.loaders import (
    get_mimi as get_jax_mimi,
)

device = torch.get_default_device()
mimi_weight = hf_hub_download(
    "kyutai/moshiko-pytorch-bf16", "tokenizer-e351c8d8-checkpoint125.safetensors"
)
model = get_mimi(mimi_weight, device='cpu')
model.set_num_codebooks(8)
jax_model = get_jax_mimi("pouet", jax.random.key(1))

/home/tugdual/miniconda3/envs/xtts/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


25.0
12.5


In [2]:


# print(their_params.keys())

In [3]:
jax.tree_util.tree_map_with_path(lambda p, _: print(jax.tree_util.keystr(p)), jax_model)

.encoder.channels
.encoder.dimension
.encoder.n_filters
.encoder.ratios[0]
.encoder.ratios[1]
.encoder.ratios[2]
.encoder.ratios[3]
.encoder.n_residual_layers
.encoder.hop_length
.encoder.n_blocks
.encoder.disable_norm_outer_blocks
.encoder.act
.encoder.first_layer.conv.conv.weight
.encoder.first_layer.conv.conv.bias
.encoder.first_layer.conv.norm_type
.encoder.first_layer.causal
.encoder.first_layer.pad_mode
.encoder.blocks[0][0][0].act
.encoder.blocks[0][0][0].blocks[0].conv.conv.weight
.encoder.blocks[0][0][0].blocks[0].conv.conv.bias
.encoder.blocks[0][0][0].blocks[0].conv.norm_type
.encoder.blocks[0][0][0].blocks[0].causal
.encoder.blocks[0][0][0].blocks[0].pad_mode
.encoder.blocks[0][0][0].blocks[1].conv.conv.weight
.encoder.blocks[0][0][0].blocks[1].conv.conv.bias
.encoder.blocks[0][0][0].blocks[1].conv.norm_type
.encoder.blocks[0][0][0].blocks[1].causal
.encoder.blocks[0][0][0].blocks[1].pad_mode
.encoder.blocks[0][0][0].true_skip
.encoder.blocks[0][1].conv.conv.weight
.encoder

MimiModel(
  encoder=SEANetEncoder(
    channels=None,
    dimension=None,
    n_filters=None,
    ratios=[None, None, None, None],
    n_residual_layers=None,
    hop_length=None,
    n_blocks=None,
    disable_norm_outer_blocks=None,
    act=None,
    first_layer=StreamingConv1d(
      conv=NormConv1d(
        conv=Conv1d(
          num_spatial_dims=1,
          weight=None,
          bias=None,
          in_channels=1,
          out_channels=64,
          kernel_size=(7,),
          stride=(1,),
          padding=((0, 0),),
          dilation=(1,),
          groups=1,
          use_bias=True,
          padding_mode='ZEROS'
        ),
        norm_type=None
      ),
      causal=None,
      pad_mode=None
    ),
    blocks=[
      (
        [
          SEANetResnetBlock(
            act=None,
            blocks=[
              StreamingConv1d(
                conv=NormConv1d(
                  conv=Conv1d(
                    num_spatial_dims=1,
                    weight=None,
      

In [4]:
import jax.tree_util as jtu
import jax.numpy as jnp

their_params = {key: jax.numpy.array(numpy.array(value.detach())) for key, value in model.encoder_transformer.named_parameters()}

def blop_trans(path, x):
    path = jtu.keystr(path)[1:]
    if "encoder_transformer.transformer.layers[" in path:
        # print(path)
        if "weights_per_step" in path:
            return x
        elif "weight" in path or "bias" in path or ".scale" in path:
            path = path.replace("encoder_transformer.", "")
            path = path.replace("in_proj.weight", "in_proj_weight")
            path = path.replace("[", ".").replace("]", "")     
            return their_params[path]
    
    return x

jax_model = jtu.tree_map_with_path(blop_trans, jax_model)

their_params = {key: jax.numpy.array(numpy.array(value.detach())) for key, value in model.decoder_transformer.named_parameters()}

def blop_de_trans(path, x):
    path = jtu.keystr(path)[1:]
    # print(path)
    if "decoder_transformer.transformer.layers[" in path:
        if "weights_per_step" in path:
            return x
        elif "weight" in path or "bias" in path or ".scale" in path:
            # print(path)
            path = path.replace("decoder_transformer.", "")
            path = path.replace("in_proj.weight", "in_proj_weight")
            path = path.replace("[", ".").replace("]", "")     
            return their_params[path]
    print(path)
    return x

jax_model = jtu.tree_map_with_path(blop_de_trans, jax_model)

their_params = {key: jax.numpy.array(numpy.array(value.detach())) for key, value in model.quantizer.named_parameters()}

def quant_blop(path, x):
    path = jtu.keystr(path)[1:]
    if "quantizer" not in path:
        return x
    path = path.replace("quantizer.", "")
    if path in their_params.keys():
        return their_params[path]
    if path == "rvq_first.vq.layers[0]._codebook.embedding":
        print(path)
        return jnp.array(
            numpy.array(model.quantizer.rvq_first.vq.layers[0]._codebook.embedding)
        )
    if ".vq.layers[" in path and "]._codebook.embedding" in path:
        idx =int( path.split("[")[1][0])
        print(path)
        return jnp.array(
            numpy.array(model.quantizer.rvq_rest.vq.layers[idx]._codebook.embedding)
        )
    return x


jax_model = jtu.tree_map_with_path(quant_blop, jax_model)


our_x = jax.random.normal(jax.random.key(1), shape=(1, 512, 10))
their_x = torch.from_numpy(numpy.array(our_x))

their_result = model.quantizer(their_x, 10)
result = jax.vmap(jax_model.quantizer, in_axes=(0, None))(our_x, 10)

print(their_result)
print(result)

# our_x = jax.random.normal(jax.random.key(1), shape=(1, 512, 101))
# their_x= torch.Tensor(numpy.array(our_x))

# our_res = jax.vmap(jax_model.decoder_transformer)(our_x)
# their_res = model.decoder_transformer(their_x)

# print(our_res[0][0,1,:10])
# print(their_res[0][0,1,:10])


encoder.channels
encoder.dimension
encoder.n_filters
encoder.ratios[0]
encoder.ratios[1]
encoder.ratios[2]
encoder.ratios[3]
encoder.n_residual_layers
encoder.hop_length
encoder.n_blocks
encoder.disable_norm_outer_blocks
encoder.act
encoder.first_layer.conv.conv.weight
encoder.first_layer.conv.conv.bias
encoder.first_layer.conv.norm_type
encoder.first_layer.causal
encoder.first_layer.pad_mode
encoder.blocks[0][0][0].act
encoder.blocks[0][0][0].blocks[0].conv.conv.weight
encoder.blocks[0][0][0].blocks[0].conv.conv.bias
encoder.blocks[0][0][0].blocks[0].conv.norm_type
encoder.blocks[0][0][0].blocks[0].causal
encoder.blocks[0][0][0].blocks[0].pad_mode
encoder.blocks[0][0][0].blocks[1].conv.conv.weight
encoder.blocks[0][0][0].blocks[1].conv.conv.bias
encoder.blocks[0][0][0].blocks[1].conv.norm_type
encoder.blocks[0][0][0].blocks[1].causal
encoder.blocks[0][0][0].blocks[1].pad_mode
encoder.blocks[0][0][0].true_skip
encoder.blocks[0][1].conv.conv.weight
encoder.blocks[0][1].conv.conv.bias
en

In [5]:
model.decoder_transformer.transformer.layers[0].self_attn.weights_per_step
# jax_model.decoder_transformer.transformer.layers[0].self_attn.weights_per_step


0

In [6]:
from moshi_jax.moshi_jax.modules.rope import RotaryEmbedding as JAXEmb
from moshi.moshi.modules.rope import RotaryEmbedding
from functools import partial

our_x = jax.numpy.ones(shape=(1, 1, 512, 10))
their_x = torch.from_numpy(numpy.array(our_x))

their_result = RotaryEmbedding()(q=their_x, k=their_x, offset=torch.zeros([1]))
pouet = partial(JAXEmb().__call__, offset=0)
result = jax.vmap(pouet, in_axes=(0,0))(our_x[0], our_x[0])

print(their_result)
print(result)



# print(their_params.keys())



Shape for rotary embeddings: (512, 10)
Shape of ts: (1, 512, 1)
(tensor([[[[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [-0.3012,  1.3818,  0.8296,  ...,  1.0040,  0.9994,  1.0006],
          [-1.3254,  0.4932,  0.6385,  ...,  1.0079,  0.9987,  1.0013],
          ...,
          [ 0.9361,  1.0600,  1.3786,  ...,  0.4580,  0.6332,  1.2645],
          [-0.3862,  1.3605,  1.4111,  ...,  0.4527,  0.6324,  1.2649],
          [-1.3534,  0.4101,  1.4082,  ...,  0.4474,  0.6316,  1.2653]]]]), tensor([[[[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [-0.3012,  1.3818,  0.8296,  ...,  1.0040,  0.9994,  1.0006],
          [-1.3254,  0.4932,  0.6385,  ...,  1.0079,  0.9987,  1.0013],
          ...,
          [ 0.9361,  1.0600,  1.3786,  ...,  0.4580,  0.6332,  1.2645],
          [-0.3862,  1.3605,  1.4111,  ...,  0.4527,  0.6324,  1.2649],
          [-1.3534,  0.4101,  1.4082,  ...,  0.4474,  0.6316,  1.2653]]]]))
(Array([[[ 1.        ,  1.       

In [7]:
result[0].dtype

dtype('float32')

In [8]:

their_params = {key: jax.numpy.array(numpy.array(value.detach())) for key, value in model.encoder.named_parameters()}

mapping = {
    "first_layer.conv.conv.weight": "model.0.conv.conv.weight",
    "first_layer.conv.conv.bias": "model.0.conv.conv.bias",
    "last_layer.conv.conv.weight": "model.14.conv.conv.weight",
    "last_layer.conv.conv.bias": "model.14.conv.conv.bias",
}

def enc_blop(path, x):
    path = jtu.keystr(path)
    if ".encoder." not in path:
        return x
    path = path.replace(".encoder.", "")    # print(path)
    if path in mapping.keys():
        return their_params[mapping[path]] if "weight" in path else jnp.expand_dims(their_params[mapping[path]], -1)
    if "weight" in path or "bias" in path:
        if path[10:11] == "1":
            idx = (int(path[7:8]) + 1) * 3

            return their_params[f"model.{idx}.conv.conv.weight"] if "weight" in path else jnp.expand_dims(their_params[f"model.{idx}.conv.conv.bias"],-1)
        else:
            first_idx = 1 + (int(path[7:8])* 3)
            second_idx = (int(path.split("blocks[")[2][0]) * 2) + 1
            return their_params[f"model.{first_idx}.block.{second_idx}.conv.conv.weight"] if "weight" in path else jnp.expand_dims(their_params[f"model.{first_idx}.block.{second_idx}.conv.conv.bias"], -1)
    return x

jax_model = jtu.tree_map_with_path(enc_blop, jax_model)

their_params = {key: jax.numpy.array(numpy.array(value.detach())) for key, value in model.decoder.named_parameters()}

mapping = {
    "first_layer.conv.conv.weight": "model.0.conv.conv.weight",
    "first_layer.conv.conv.bias": "model.0.conv.conv.bias",
    "last_layer.conv.conv.weight": "model.14.conv.conv.weight",
    "last_layer.conv.conv.bias": "model.14.conv.conv.bias",
}

def dec_blop(path, x):
    path = jtu.keystr(path)
    if ".decoder." not in path:
        return x
    print(path)
    path = path.replace(".decoder.", "")    # print(path)
    if path in mapping.keys():
        print(path)
        return their_params[mapping[path]] if "weight" in path else jnp.expand_dims(their_params[mapping[path]], -1)
    if "weight" in path or "bias" in path:
        if path[10:11] == "0":
            idx = 2 + (int(path[7:8])* 3)
            print(path)
            rest = path.split("]")[-1]
            return jnp.swapaxes(jnp.flip(their_params[f"model.{idx}{rest}"], axis=2),axis1=0, axis2=1) if "convtr.weight" in path else their_params[f"model.{idx}{rest}"] if "conv.weight" in path else jnp.expand_dims(their_params[f"model.{idx}.convtr.convtr.bias"],-1)
        else:
            first_idx = 3 + (int(path[7:8])* 3)
            second_idx = (int(path.split("blocks[")[2][0]) * 2) + 1
            print(path)
            rest = path.split("]")[-1]
            
            return jnp.swapaxes(jnp.flip(their_params[f"model.{first_idx}.block.{second_idx}{rest}"], axis=2), axis1=0, axis2=1) if "convtr.weight" in path else their_params[f"model.{first_idx}.block.{second_idx}{rest}"] if "weight" in path else jnp.expand_dims(their_params[f"model.{first_idx}.block.{second_idx}{rest}"], -1)
    return x

jax_model = jtu.tree_map_with_path(dec_blop, jax_model)

their_params = {key: jax.numpy.array(numpy.array(value.detach())) for key, value in model.named_parameters()}


def blop_model(path, x):
    path = jtu.keystr(path)
    print(path)
    if ".upsample.convtr.convtr.convtr.weight" == path:
        return jnp.flip(their_params["upsample.convtr.convtr.convtr.weight"], axis=2)
    if ".downsample.conv.conv.conv.weight" == path:
        return their_params["downsample.conv.conv.conv.weight"]
    return x    

    # return x÷

jax_model = jtu.tree_map_with_path(blop_model, jax_model)



# our_x = jax.random.normal(jax.random.key(1), shape=(1, 1, 960))
# their_x= torch.Tensor(numpy.array(our_x))

# their_res = model.encoder(their_x)
# our_res = jax.vmap(jax_model.encoder)(our_x)
# their_res= torch.Tensor(numpy.array(our_res))

# print(our_res[0, :3])
# print(their_res[0, :3])



.decoder.channels
.decoder.dimension
.decoder.n_filters
.decoder.ratios[0]
.decoder.ratios[1]
.decoder.ratios[2]
.decoder.ratios[3]
.decoder.n_residual_layers
.decoder.hop_length
.decoder.n_blocks
.decoder.disable_norm_outer_blocks
.decoder.act
.decoder.first_layer.conv.conv.weight
first_layer.conv.conv.weight
.decoder.first_layer.conv.conv.bias
first_layer.conv.conv.bias
.decoder.first_layer.conv.norm_type
.decoder.first_layer.causal
.decoder.first_layer.pad_mode
.decoder.blocks[0][0].convtr.convtr.weight
blocks[0][0].convtr.convtr.weight
.decoder.blocks[0][0].convtr.convtr.bias
blocks[0][0].convtr.convtr.bias
.decoder.blocks[0][0].convtr.norm_type
.decoder.blocks[0][0].causal
.decoder.blocks[0][0].trim_right_ratio
.decoder.blocks[0][1][0].act
.decoder.blocks[0][1][0].blocks[0].conv.conv.weight
blocks[0][1][0].blocks[0].conv.conv.weight
.decoder.blocks[0][1][0].blocks[0].conv.conv.bias
blocks[0][1][0].blocks[0].conv.conv.bias
.decoder.blocks[0][1][0].blocks[0].conv.norm_type
.decoder.

In [9]:
our_x = jax.random.normal(jax.random.key(1), shape=(1, 1, 960))
their_x= torch.Tensor(numpy.array(our_x))


In [10]:
our_x = jax.random.normal(jax.random.key(1), shape=(1, 512, 40))
their_x= torch.Tensor(numpy.array(our_x))
# print(model.decoder.model)
# print(model.decoder.model[2].convtr.convtr.weight[0, :3])
# print(jax_model.decoder.blocks[0][0].convtr.convtr.weight[0, :3])
their_res = model.decoder(their_x)
our_res = jax.vmap(jax_model.decoder)(our_x)

print(our_res[0, 0, :5])
print(their_res[0, 0, :5])


[0.01198527 0.01390748 0.00132943 0.00722347 0.016158  ]
tensor([0.0120, 0.0139, 0.0013, 0.0072, 0.0162], grad_fn=<SliceBackward0>)


In [26]:
import jax
import librosa
import jax.numpy as jnp
y, sr = librosa.load("bria.mp3")
if sr != 24000:
    y = librosa.resample(y, orig_sr=sr, target_sr=24000)

y = jnp.array(y[None, None, :240000])
y2 = torch.Tensor(numpy.array(y))


In [21]:
print(model.downsample.conv.conv.conv.weight[0,0])
print(jax_model.downsample.conv.conv.conv.weight[0,0])

print(jax.vmap(jax_model.downsample)(out[0])[0, 0, :10])
print(model.downsample(torch.from_numpy(numpy.array(out[0])))[0,0, :10])

tensor([-0.0654, -0.1083,  0.3409, -0.3511], grad_fn=<SelectBackward0>)
[-0.06542415 -0.10834397  0.3409385  -0.35108143]
[ 1.5655878  -0.52208686 -1.6100225  -0.32830822 -0.68118566 -1.3413008
 -0.86556995 -0.88997257 -1.3677669   0.02973923]
tensor([ 1.5662, -0.5222, -1.6101, -0.3282, -0.6812, -1.3417, -0.8660, -0.8900,
        -1.3679,  0.0294], grad_fn=<SliceBackward0>)


In [64]:
# print(y2.dtype)

out = jax.vmap(jax_model.encoder)(y)
with torch.no_grad():
    with model._context_for_encoder_decoder:
        a = model.encoder(y2)
# out2 = model.encoder(y2)
print(out[0, 0, :10])
print(a[0, 0, :10])

b = model.encoder_transformer(a)
out = jax.vmap(jax_model.encoder_transformer)(out)

print(out[0][0, 0, :10])
print(b[0][0, 0, :10])
print(out[0].shape)
c = model._to_framerate(b[0])
out = (jax.vmap(jax_model._to_framerate)(out[0]), out[1:])

print(out[0][0, 0, :10])
print(c[0, 0, :10])


[-0.01047584 -0.003635    0.041963    0.00060157  0.00288849 -0.00126958
 -0.00375559 -0.00125686 -0.01063192  0.01241607]
tensor([-0.0105, -0.0036,  0.0420,  0.0006,  0.0029, -0.0013, -0.0038, -0.0013,
        -0.0106,  0.0124])
T State of attention : tensor([[-0.0106,  0.0246, -0.0049,  0.0146, -0.0034],
        [-0.0037, -0.0090,  0.0072,  0.0048,  0.0100]],
       grad_fn=<SliceBackward0>)
T State of attention : tensor([[-0.0038,  0.0145,  0.0107,  0.0042,  0.0053],
        [-0.0023, -0.0130,  0.0104,  0.0073,  0.0061]],
       grad_fn=<SliceBackward0>)
T State of attention : tensor([[-0.0024,  0.0172,  0.0077,  0.0011,  0.0037],
        [-0.0018, -0.0129,  0.0061,  0.0013,  0.0093]],
       grad_fn=<SliceBackward0>)
T State of attention : tensor([[-0.0021,  0.0155,  0.0034,  0.0009, -0.0023],
        [-0.0019, -0.0155,  0.0064, -0.0017,  0.0087]],
       grad_fn=<SliceBackward0>)
T State of attention : tensor([[-0.0021,  0.0101,  0.0022,  0.0021, -0.0017],
        [-0.0019, -0.014

In [65]:

# out = jax.numpy.array(out2[0].detach().numpy())

d = model.quantizer(c, model.frame_rate)
out = jax.vmap(jax_model.quantizer, in_axes=(0, None))(out[0], jax_model.frame_rate)
# print(out2.x.shape)
print(out[0].shape)
e = model._to_encoder_framerate(d.x)
out = (jax.vmap(jax_model._to_encoder_framerate)(out[0]), out[1:])
# out = (jax.numpy.array(out2.x.detach().numpy()), out[1:])
# print(out2.x.shape)
print(out[0].shape)
# out = jax.numpy.array(out2[0].detach().numpy())

out = jax.vmap(jax_model.decoder_transformer)(out[0])
e = model.decoder_transformer(e)
# print("AAA")


T after quant: tensor([[-1.0071,  0.0838, -0.6477,  ..., -0.8159,  0.0407, -0.1958],
        [-1.4651,  0.5905, -0.0174,  ...,  1.0387, -0.4181, -0.0094],
        [-1.2467, -0.6013,  0.5687,  ..., -0.7009, -0.4161, -0.3761],
        ...,
        [-1.0087, -0.2221, -0.0054,  ..., -0.4583, -0.2306, -0.4086],
        [-1.0895,  0.0789, -0.1939,  ..., -1.0513,  0.6082,  1.0979],
        [-0.2161,  0.7345, -0.3479,  ...,  0.3993, -0.5223, -0.0837]])
T after quant: tensor([[-0.3254,  0.1711, -0.0275,  ...,  0.0844,  0.4085, -0.3901],
        [ 0.1678,  0.1822,  0.2877,  ..., -0.0991,  0.0068,  0.1726],
        [-0.0515, -0.0315, -0.0568,  ...,  0.0585,  0.1771, -0.0927],
        ...,
        [-0.2557,  0.4989,  0.0735,  ...,  0.0117, -0.1742, -0.2409],
        [ 0.2124, -0.1259, -0.1448,  ..., -0.0331,  0.3859,  0.0601],
        [-0.2666,  0.0407, -0.2333,  ...,  0.2374, -0.2061, -0.2307]])
O aaa [ 0.07764138  0.11011618 -0.9440186 ]
O after quant: [[-1.007068   -0.26334637 -0.01980272 ... -

In [66]:
print(out[0][0, 1, :10])
# print(out2.x[0, 1, :10])


[ 0.47146547 -0.03033069  0.27556014 -0.25042832 -0.2955848   1.0365841
  0.37958673 -0.07709883 -0.38753307 -0.1604212 ]


In [68]:


# out2 = torch.from_numpy(numpy.array(out[0]))
# out = jax.numpy.array(out2[0].detach().numpy())
# print(out2[0].shape)

with torch.no_grad():
    with model._context_for_encoder_decoder:
        f = model.decoder(e[0])
out = jax.vmap(jax_model.decoder)(out[0])

# print(out[0, 0, :10])
# print(out2[0, 0, :10])


In [70]:
import IPython
print(y2.shape)
# IPython.display.Audio(y2[0], rate=24000)
IPython.display.Audio(out[0][0], rate=24000)

torch.Size([1, 1, 240000])


In [48]:
IPython.display.Audio(model(y2).x.detach().numpy()[0], rate=24000)


T State of attention : tensor([[-0.0106,  0.0246, -0.0049,  0.0146, -0.0034],
        [-0.0037, -0.0090,  0.0072,  0.0048,  0.0100]],
       grad_fn=<SliceBackward0>)
T State of attention : tensor([[-0.0038,  0.0145,  0.0107,  0.0042,  0.0053],
        [-0.0023, -0.0130,  0.0104,  0.0073,  0.0061]],
       grad_fn=<SliceBackward0>)
T State of attention : tensor([[-0.0024,  0.0172,  0.0077,  0.0011,  0.0037],
        [-0.0018, -0.0129,  0.0061,  0.0013,  0.0093]],
       grad_fn=<SliceBackward0>)
T State of attention : tensor([[-0.0021,  0.0155,  0.0034,  0.0009, -0.0023],
        [-0.0019, -0.0155,  0.0064, -0.0017,  0.0087]],
       grad_fn=<SliceBackward0>)
T State of attention : tensor([[-0.0021,  0.0101,  0.0022,  0.0021, -0.0017],
        [-0.0019, -0.0147,  0.0076,  0.0052,  0.0041]],
       grad_fn=<SliceBackward0>)
T State of attention : tensor([[-0.0021,  0.0078, -0.0034,  0.0028, -0.0020],
        [-0.0019, -0.0184, -0.0019,  0.0078,  0.0040]],
       grad_fn=<SliceBackward0>

In [ ]:
print(out2.codes)

In [ ]:
jax_model.resample_method